In [ ]:
from newsgac import database

In [ ]:
from newsgac.data_sources.models import DataSource
from newsgac.pipelines import Pipeline

In [ ]:
Pipeline.objects.count()

In [ ]:
pl = Pipeline.objects.all()[36]

In [17]:
def visualise_results(pipeline_id):
    # TODO: refactor
    pipeline = Pipeline.objects.get({'_id': ObjectId(pipeline_id)})
    sk_pipeline = pipeline.sk_pipeline.get()
    classifier = sk_pipeline.named_steps['Classifier']
    results_eval = pipeline.result
    results_model = pipeline.result
    p, script, div = ResultVisualiser.retrieveHeatMapfromResult(normalisation_flag=True, result=results_eval, title="Evaluation", ds_param=0.7)
    # p_mod, script_mod, div_mod = ResultVisualiser.retrieveHeatMapfromResult(normalisation_flag=True, result=results_model, title="Model", ds_param=0.7)
    script_f = ''
    div_f = ''
    # TODO: check for nb
    # TODO: _svc because it's crashing atm
    if pipeline.learner._tag in ['_svc']:
        if classifier.kernel == 'linear':
            coefficients = classifier.coef_
            vectorized_pipeline = sk_pipeline.named_steps['FeatureExtraction'].transformer_list[0][0] == 'TFIDF'
            names = sk_pipeline.named_steps['FeatureExtraction'].get_feature_names()
            # get vectorizer for bow
            if isinstance(coefficients, csr_matrix):
                coefficients = coefficients.toarray()
            p_f, script_f, div_f = ResultVisualiser.retrievePlotForFeatureWeights(coefficients=coefficients, names=names, vectorized_pipeline=vectorized_pipeline )
    elif pipeline.learner.tag in ['xgb']:
        from pandas import DataFrame
        from collections import OrderedDict

        feature_weights = classifier.get_booster().get_fscore()
        sorted_fw = OrderedDict(sorted(feature_weights.items(), key=lambda t: t[0]))
        sorted_keys = sorted(sk_pipeline.named_steps['FeatureExtraction'].get_feature_names())
        print sorted_fw

        feat_importances = []
        for (ft, score) in sorted_fw.items():
            index = int(ft.split("f")[1])
            feat_importances.append({'Feature': sorted_keys[index], 'Importance': score})
        feat_importances = DataFrame(feat_importances)
        feat_importances = feat_importances.sort_values(
            by='Importance', ascending=True).reset_index(drop=True)
        # Divide the importances by the sum of all importances
        # to get relative importances. By using relative importances
        # the sum of all importances will equal to 1, i.e.,
        # np.sum(feat_importances['importance']) == 1
        feat_importances['Importance'] /= feat_importances['Importance'].sum()
        feat_importances = feat_importances.round(3)

        p_f, script_f, div_f = ResultVisualiser.visualize_df_feature_importance(feat_importances, pipeline.display_title)
    elif pipeline.learner.tag in ['rf']:
        from pandas import DataFrame
        feature_weights = classifier.feature_importances_
        sorted_keys = sorted(sk_pipeline.named_steps['FeatureExtraction'].get_feature_names())

        feat_importances = []
        for (ft, key) in zip(feature_weights, sorted_keys):
            feat_importances.append({'Feature': key, 'Importance': ft})
        feat_importances = DataFrame(feat_importances)
        feat_importances = feat_importances.sort_values(
            by='Importance', ascending=True).reset_index(drop=True)
        # Divide the importances by the sum of all importances
        # to get relative importances. By using relative importances
        # the sum of all importances will equal to 1, i.e.,
        # np.sum(feat_importances['importance']) == 1
        feat_importances['Importance'] /= feat_importances['Importance'].sum()
        feat_importances = feat_importances.round(3)

        p_f, script_f, div_f = ResultVisualiser.visualize_df_feature_importance(feat_importances,
                                                                                pipeline.display_title)

    plots = []
    plots.append(p)
    overview_layout = gridplot(plots, ncols=2)
    script, div = components(overview_layout)

    return render_template('pipelines/results.html',
                           pipeline=pipeline,
                           results_eval=results_eval,
                           results_model=results_model,
                           plot_script=script,
                           plot_div=div,
                           plot_feature_script=script_f,
                           plot_feature_div=div_f,
                           mimetype='text/html',
                           mermaid=get_mermaid(pipeline.get_sk_pipeline())
)

<DataSource object>
